In [2]:
import numpy as np
import scipy.stats as sts
import scipy.optimize as opt

In [3]:
data = np.loadtxt('../data/MacroSeries.txt', delimiter=',')

In [7]:
c = data[:, 0]
k = data[:, 1]
w = data[:, 2]
r = data[:, 3]
beta = 0.99

In [6]:
def get_z_r(alpha, k, r):
    n = k.shape[0]
    z = np.empty(n)
    for i in range(n):
        z[i] = np.log(r[i] / (alpha * (k[i] ** (alpha - 1))))
    return z

In [8]:
def get_expectations(alpha, beta, rho, mu, c, k, w, r):
    z = get_z_r(alpha, k, r)
    n = z.shape[0]
    e1_vec = np.empty(n - 1)
    e2_vec = np.empty(n - 1)
    e3_vec = np.empty(n - 1)
    e4_vec = np.empty(n - 1)
    for i in range(n - 1):
        e1_vec[i] = z[i + 1] - rho*z[i] - (1 - rho)*mu
        e2_vec[i] = e1_vec[i]*z[i]
        e3_vec[i] = beta*alpha*np.exp(z[i + 1])*(k[i + 1] ** (alpha - 1))*(c[i] / c[i + 1]) - 1
        e4_vec[i] = e3_vec[i]*w[i]
    return e1_vec.mean(), e2_vec.mean(), e3_vec.mean(), e4_vec.mean()

In [9]:
def crit(params, c, k, w, r, W):
    alpha, beta, rho, mu = params
    e1, e2, e3, e4 = get_expectations(alpha, beta, rho, mu, c, k, w, r)
    err_vec = np.array([e1, e2, e3, e4])
    crit_val = np.dot(np.dot(err_vec.T, W), err_vec)
    return crit_val

In [28]:
W = np.identity(4)
alpha_init = 0.4
beta_init = 0.2
rho_init = 0.4
mu_init = 3
params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
results = opt.minimize(crit, params_init, args=(c, k, w, r, W), bounds=((1e-10, 1), (1e-10, 1), (-1, 1), (1e-10, None)))
alpha_GMM, beta_GMM, rho_GMM, mu_GMM = results.x
print('alpha_GMM: ', alpha_GMM, 'beta_GMM: ', beta_GMM, 'rho_GMM: ', rho_GMM, 'mu_GMM: ', mu_GMM)
print('Value of minimized crit function: ', crit((alpha_GMM, beta_GMM, rho_GMM, mu_GMM), c, k, w, r, W))

alpha_GMM:  0.721932899517 beta_GMM:  0.989999995001 rho_GMM:  0.999961683708 mu_GMM:  3.02443780625
Value of minimized crit function:  0.0027675262016
